In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
NUMBER_OF_DAYS = 100
EVENT_WINDOW_PRE = 2
EVENT_WINDOW_POST = 3
EVENT_DAY = 70
np.random.seed(0)

def generate_jumping_stock(n_days):
    noise_amp = 10
    days = np.arange(0, n_days)
    stock_price = noise_amp * np.random.normal(size=n_days)
    for i in range(n_days):
        if i < EVENT_DAY:
            stock_price[i] = stock_price[i] + 200 + i * 2
        else:
            stock_price[i] = stock_price[i] + 400 + i * 2
    return days, stock_price

def generate_returns(time_series):
    return_series = []
    for i in range(len(time_series) - 1):
        return_series.append(1 - time_series[i] / time_series[i + 1])
    return_series.append(0)
    return np.array(return_series)

def CMR_model(time, returns):
    """
    Returns the mean and Variance of the returns
    """
    return np.average(returns), np.var(returns)

In [3]:
time, series = generate_jumping_stock(NUMBER_OF_DAYS)
returns = generate_returns(series)
cmr_mean, cmr_var = CMR_model(time, returns)
print("CMR mean: ", cmr_mean)
print("CMR variance: ", cmr_var)

abnormal_returns = returns - cmr_mean

# calculate average abnormal returns on event window
avg_abnormal_returns = np.mean(abnormal_returns[EVENT_DAY - EVENT_WINDOW_PRE:EVENT_DAY + EVENT_WINDOW_POST])
print("Average abnormal returns: ", avg_abnormal_returns)

CMR mean:  0.008090261960032694
CMR variance:  0.0035526525690920373
Average abnormal returns:  0.06978870360682607


In [4]:
# find the z score
import scipy.stats as stats
z_score = avg_abnormal_returns / np.sqrt(cmr_var / (EVENT_WINDOW_PRE + EVENT_WINDOW_POST))
print("Z score: ", z_score)
p_value = 1 - stats.norm.cdf(z_score)
print("P value: ", p_value)

Z score:  2.6181454154804915
P value:  0.004420456054424982


In [5]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=series, mode='lines', name='Stock price'))
fig.add_vrect(x0=EVENT_DAY - EVENT_WINDOW_PRE, x1=EVENT_DAY + EVENT_WINDOW_POST, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)
fig.update_layout(
    title="Event study",
    xaxis_title="Days",
    yaxis_title="Stock price",
    showlegend=True
)

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=returns, mode='lines', name='Returns'))
fig.add_vrect(x0=EVENT_DAY - EVENT_WINDOW_PRE, x1=EVENT_DAY + EVENT_WINDOW_POST, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)
fig.update_layout(
    title="Event study",
    xaxis_title="Days",
    yaxis_title="Returns",
    showlegend=True
)

In [8]:
fig = go.Figure()
abnormal_returns_mva = np.convolve(abnormal_returns, np.ones((EVENT_WINDOW_PRE + EVENT_WINDOW_POST,))/EVENT_WINDOW_PRE, mode='valid')
fig.add_trace(go.Scatter(x=time, y=abnormal_returns, mode='lines', name='Abnormal Returns'))
fig.add_trace(go.Scatter(x=time[EVENT_WINDOW_PRE: -EVENT_WINDOW_POST], y=abnormal_returns_mva, mode='lines', name='Moving Average Abnormal Returns'))
fig.add_hrect(y0=-np.sqrt(cmr_var / (EVENT_WINDOW_PRE + EVENT_WINDOW_POST))*2, y1=np.sqrt(cmr_var / (EVENT_WINDOW_PRE + EVENT_WINDOW_POST))*2, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)